In [1]:
from tensorflow.python.client import device_lib
from tensorflow.keras import layers
import gymnasium as gym
#from gym import spaces
import tensorflow as tf
import cellrank as cr
import scanpy as sc
import pandas as pd
import numpy as np
#import gym
# Training the agent with DQN
#from stable_baseline3 import DQN 

from reinforcement_learning_module import LatentSpaceEnv,SimpleDQN

%load_ext autoreload
%autoreload 2

Global seed set to 0
/Users/brendamelano/miniconda3/envs/scvi_env/lib/python3.11/site-packages/pytorch_lightning/utilities/warnings.py:53: LightningDeprecationWarning: pytorch_lightning.utilities.warnings.rank_zero_deprecation has been deprecated in v1.6 and will be removed in v1.8. Use the equivalent function from the pytorch_lightning.utilities.rank_zero module instead.
  new_rank_zero_deprecation(
/Users/brendamelano/miniconda3/envs/scvi_env/lib/python3.11/site-packages/pytorch_lightning/utilities/warnings.py:58: LightningDeprecationWarning: The `pytorch_lightning.loggers.base.rank_zero_experiment` is deprecated in v1.7 and will be removed in v1.9. Please use `pytorch_lightning.loggers.logger.rank_zero_experiment` instead.
  return new_rank_zero_deprecation(*args, **kwargs)


In [ ]:

# Reading in the action_df from svae+
action_df = pd.read_csv('/Users/brendamelano/Desktop/REPROGRAM/action_df.csv', index_col=0)

# Creating the environment based on the class that I built for cluster 5
target_point = np.array([-0.9369193 , -0.447852  , -0.45398346,  1.777766  ,  0.32149878,
       -0.60258126])


# Creating the latent space environment
env = LatentSpaceEnv(action_df=action_df, target_point=target_point)


# Initialize DQN agent parameters
alpha = 0.2  # Learning rate.. how exactly does this work algorithmically?
gamma = 0.3   # Discount factor
epsilon = 0.1  # Exploration rate


# Create DQN agent
dqn_agent = SimpleDQN(observation_space=env.observation_space,
                      action_space=env.action_space,
                      action_df=action_df,  # Pass action_df to the agent
                      alpha=alpha,
                      gamma=gamma,
                      epsilon=epsilon)

# Training loop parameters
num_episodes = 20
max_steps_per_episode = 100


# Training loop
for episode in range(num_episodes):
    
    # Reset the environment at the start of each episode
    state = env.reset()
    total_reward = 0
    
    for step in range(max_steps_per_episode):
        
        # Choose an action
        action = dqn_agent.choose_action(state)
        
        # Take action in the environment
        next_state, reward, done, _ = env.step(action)
        
        # Learn from the experience
        dqn_agent.learn(state, action, reward, next_state, done)
        
        # Update the current state
        state = next_state
        
        # Accumulate the reward
        total_reward += reward
        
        # End the episode if done
        if done:
            break
    
    # Print episode statistics
    print(f"Episode {episode + 1}/{num_episodes}, Total Reward: {total_reward}, Steps: {step + 1}")

print("Training completed.")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


In [7]:
# Test the agent with initial state from cluster 2
initial_state = np.array([0.5738455 ,  0.6803657 ,  0.52349895, -0.96093357, -1.05216   ,
       -0.6411334])

target_state = env.target_point
perturbations = dqn_agent.get_perturbations_to_target(initial_state, target_state)

print("Perturbations to target:", perturbations)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step


In [8]:
number_list = [290, 144, 389, 487, 378, 651, 627, 651, 627, 318, 597, 51, 336, 518, 336, 93, 93, 93, 597, 93, 93, 29, 438, 438, 453, 93, 93, 93, 93, 93, 135, 513, 432, 184, 501, 501, 518, 518, 518, 317, 596, 317, 518, 317, 596, 317, 501, 178, 501, 178]

# Extract unique values from the list
unique_numbers = list(set(number_list))


# Assuming action_df is already read in from a CSV or created
# action_df = pd.read_csv('/path/to/action_df.csv')


# Select the rows from action_df based on the unique indices
selected_rows = action_df.iloc[unique_numbers]


# Print the selected rows
print(selected_rows)

         Latent_0  Latent_1  Latent_2  Latent_3  Latent_4  Latent_5
RPS15A  -0.692921 -0.357513 -0.164269  0.723772  0.072576 -0.145721
PDCD11  -1.268080 -0.599613 -0.773501  2.129370  0.124935 -0.592556
RPS23   -0.676829 -0.346698 -0.315413  0.684083  0.071434 -0.160900
EIF3G   -0.370982 -0.543809  0.074646  0.081183  0.067712 -0.865696
VPS28    0.792665  0.492349  0.524672  0.014304  0.117679 -0.236076
ENO1     0.011491  0.100250 -0.177479 -1.201127  0.224771  0.314871
BCAS2    0.086486  0.084760 -0.008747 -0.487085  0.418158 -0.526808
MED21    0.819884  0.587833 -0.093482  0.792575  0.492456  0.687769
PRPF19  -0.115619  0.085245 -0.003076 -0.230626  0.314355 -0.272154
GEMIN5   0.100848  1.087689  1.022816 -0.366746  0.707550 -0.194187
CCDC86  -0.427463 -0.264937 -0.044944  0.725017  0.375757 -1.027535
PSMD2    0.214064  0.412642  0.142055  0.433169  0.410455 -0.000469
GINS2    0.772059  0.034246  0.216064 -0.621370  0.690350  0.926288
MRPL16   0.388170  0.186877  0.272866 -1.147392 

In [10]:
selected_rows = action_df[action_df.index.isin(unique_numbers)]

# Print the selected rows
print(selected_rows)

NameError: name 'unique_numbers' is not defined